In [4]:
from spotify_functions import *
from get_album_counts import *

get_access_token()

artist_name = "Beck"
artist_id = get_artist_id(artist_name, access_token)
if artist_id:
    album_count, total_tracks = get_album_counts(artist_id, access_token)
    print(f"Artist: {artist_name}")
    print(f"Number of Albums: {album_count}")
    print(f"Total Number of Tracks: {total_tracks}")
else:
    print(f"Artist {artist_name} not found.")

KeyboardInterrupt: 

In [5]:
# language: python
import json
import requests

response = requests.get(url, headers=headers, params=params)

# raise for non-2xx statuses (will raise requests.HTTPError)
try:
    response.raise_for_status()
except requests.HTTPError:
    # handle or re-raise after logging response.status_code / response.text
    raise

# quick debug info (optional)
# print(response.status_code, response.headers.get('content-type'), response.text[:200])

# avoid calling .json() blindly
text = response.text
content_type = response.headers.get('content-type', '')

if not text:
    # empty body -> return empty result or raise a clear error
    return []

if 'application/json' not in content_type.lower():
    # non-json response -> log and handle
    raise ValueError(f'Expected JSON response, got: {content_type}')

# finally parse safely
try:
    data = response.json()
except json.JSONDecodeError:
    # response text was not valid JSON
    raise ValueError('Response contained invalid JSON') from None

# use data.get('items', [])

NameError: name 'url' is not defined

In [7]:
# language: python
# Replace the broken manual-request cell with this.

from spotify_functions import get_artist_albums

# ensure the notebook has the needed variables from the first cell
if 'artist_id' not in globals() or 'access_token' not in globals():
    raise RuntimeError("`artist_id` or `access_token` not defined. Run the cell that sets them first.")

try:
    albums = get_artist_albums(artist_id, access_token)
except Exception as exc:
    # surface any error from the helper function
    print("Error fetching albums:", exc)
    albums = []

print(f"Fetched {len(albums)} albums")
for album in albums[:10]:
    print(album.get('name'), "-", album.get('id'))

Fetched 0 albums


In [8]:
# python
import requests
import json

def get_artist_albums(artist_id, access_token, include_groups="album,single", limit=50):
    """
    Fetch artist albums from Spotify API with robust response checks.
    Returns list of album dicts or raises RuntimeError with diagnostic text.
    """
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"include_groups": include_groups, "limit": limit}

    resp = requests.get(url, headers=headers, params=params, timeout=10)

    # Raise for non-2xx and include body snippet for diagnostics
    try:
        resp.raise_for_status()
    except requests.HTTPError as exc:
        body_snip = (resp.text or "")[:500]
        content_type = resp.headers.get("content-type", "")
        raise RuntimeError(f"Spotify API HTTP {resp.status_code}: {body_snip} (content-type: {content_type})") from exc

    # Empty body -> return empty list (no albums)
    if not resp.text:
        return []

    # Ensure response is JSON-ish
    content_type = resp.headers.get("content-type", "")
    if "application/json" not in content_type.lower():
        body_snip = resp.text[:500]
        raise RuntimeError(f"Expected JSON response, got {content_type}. Body (truncated): {body_snip}")

    # Parse JSON safely
    try:
        data = resp.json()
    except json.JSONDecodeError as exc:
        body_snip = resp.text[:500]
        raise RuntimeError(f"Invalid JSON from Spotify API. Body (truncated): {body_snip}") from exc

    return data.get("items", [])

#### New Test Code

In [9]:
# python
import requests
import json
import time
from typing import List

def get_artist_albums(artist_id: str,
                      access_token: str,
                      include_groups: str = "album,single",
                      limit: int = 50,
                      retries: int = 3,
                      timeout: float = 10.0,
                      debug: bool = False) -> List[dict]:
    """
    Robustly fetch artist albums from Spotify API.
    Update `spotify/spotify_functions.py` with this function.
    """
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"include_groups": include_groups, "limit": limit}

    backoff = 1.0
    for attempt in range(1, retries + 1):
        try:
            resp = requests.get(url, headers=headers, params=params, timeout=timeout)
        except requests.exceptions.RequestException as exc:
            if debug:
                print(f"[debug] request exception (attempt {attempt}): {exc}")
            if attempt == retries:
                raise RuntimeError(f"Network error fetching albums: {exc}") from exc
            time.sleep(backoff)
            backoff *= 2
            continue

        status = resp.status_code
        content_type = resp.headers.get("content-type", "<none>")
        body_snip = (resp.text or "")[:1000]

        if debug:
            print(f"[debug] status={status} content-type={content_type} body_snip={body_snip[:200]!r}")

        # Clear messages for common error codes
        if status == 401:
            raise RuntimeError("401 Unauthorized: access token may be missing, invalid, or expired. Refresh the token.")
        if status == 429:
            retry_after = resp.headers.get("Retry-After", "unknown")
            raise RuntimeError(f"429 Rate limited. Retry-After: {retry_after}. Body (truncated): {body_snip}")
        if not resp.ok:
            raise RuntimeError(f"HTTP {status} error. content-type: {content_type}. Body (truncated): {body_snip}")

        # Empty or whitespace-only body -> treat as no items
        if not resp.text or not resp.text.strip():
            if debug:
                print(f"[debug] Empty response body (status {status}). Returning empty list.")
            return []

        # Ensure JSON-like response
        if "application/json" not in content_type.lower():
            raise RuntimeError(f"Expected JSON response but got {content_type}. Body (truncated): {body_snip}")

        # Parse JSON safely
        try:
            data = resp.json()
        except json.JSONDecodeError as exc:
            raise RuntimeError(f"Invalid JSON response from Spotify API. Body (truncated): {body_snip}") from exc

        # Normal shape: dict with 'items'
        if isinstance(data, dict):
            return data.get("items", [])
        # Unexpected JSON (but valid) -> surface for debugging
        raise RuntimeError(f"Unexpected JSON shape: {type(data)}. Body (truncated): {body_snip}")

    raise RuntimeError("Exceeded retries fetching artist albums.")

In [11]:
# Run this test code after updating the function above.
from spotify_functions import get_artist_albums
if 'artist_id' not in globals() or 'access_token' not in globals():
    raise RuntimeError("`artist_id` or `access_token` not defined. Run the cell that sets them first.")
try:
    albums = get_artist_albums(artist_id, access_token, debug=True)
except Exception as exc:
    print("Error fetching albums:", exc)
else:
    print(f"Fetched {len(albums)} albums")
    for album in albums[:10]:
        print(album.get('name'), "-", album.get('id'))

Error fetching albums: get_artist_albums() got an unexpected keyword argument 'debug'


In [12]:
# language: python
import requests
import json
import time
from typing import List

def get_artist_albums(artist_id: str,
                      access_token: str,
                      include_groups: str = "album,single",
                      limit: int = 50,
                      retries: int = 3,
                      timeout: float = 10.0,
                      debug: bool = False) -> List[dict]:
    """
    Fetch artist albums from Spotify API with diagnostics, retries and optional debug output.
    """
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"include_groups": include_groups, "limit": limit}

    backoff = 1.0
    for attempt in range(1, retries + 1):
        try:
            resp = requests.get(url, headers=headers, params=params, timeout=timeout)
        except requests.exceptions.RequestException as exc:
            if debug:
                print(f"[debug] request exception (attempt {attempt}): {exc}")
            if attempt == retries:
                raise RuntimeError(f"Network error fetching albums: {exc}") from exc
            time.sleep(backoff)
            backoff *= 2
            continue

        status = resp.status_code
        content_type = resp.headers.get("content-type", "<none>")
        body_snip = (resp.text or "")[:1000]

        if debug:
            print(f"[debug] status={status} content-type={content_type} body_snip={body_snip[:200]!r}")

        if status == 401:
            raise RuntimeError("401 Unauthorized: access token may be missing, invalid, or expired. Refresh the token.")
        if status == 429:
            retry_after = resp.headers.get("Retry-After", "unknown")
            raise RuntimeError(f"429 Rate limited. Retry-After: {retry_after}. Body (truncated): {body_snip}")
        if not resp.ok:
            raise RuntimeError(f"HTTP {status} error. content-type: {content_type}. Body (truncated): {body_snip}")

        if not resp.text or not resp.text.strip():
            if debug:
                print(f"[debug] Empty response body (status {status}). Returning empty list.")
            return []

        if "application/json" not in content_type.lower():
            raise RuntimeError(f"Expected JSON response but got {content_type}. Body (truncated): {body_snip}")

        try:
            data = resp.json()
        except json.JSONDecodeError as exc:
            raise RuntimeError(f"Invalid JSON response from Spotify API. Body (truncated): {body_snip}") from exc

        if isinstance(data, dict):
            return data.get("items", [])
        raise RuntimeError(f"Unexpected JSON shape: {type(data)}. Body (truncated): {body_snip}")

    raise RuntimeError("Exceeded retries fetching artist albums.")